# <center>Topic Modeling Using LDA (Latent Dirichlet Allocation) in Python</center>

## Importing the Required Libraries

In [19]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

#vis
#! pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from tqdm import tqdm 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Preparing the Data

In [4]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [7]:
stopwords = stopwords.words("english")

In [8]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
data = load_data("/content/drive/MyDrive/Advanced_Machine_Learning/LDA/ushmm_dn.json")["texts"]
print(len(data))
print(data[10][0:90])


252
 My name is Helene Baraf, named Zupnik. I am born in Antwerp, Belgium, the 24th of July 19


In [21]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in tqdm(texts):
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)
lemmatized_texts = lemmatization(data)
print (lemmatized_texts[10][0:90])

100%|██████████| 252/252 [02:16<00:00,  1.85it/s]

name name bear 24th start tell little bit ancestor come very famous branch jewish religion


In [22]:
def gen_words(texts):
    final = []
    for text in tqdm(texts):
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print()
print (data_words[10][0:20])

100%|██████████| 252/252 [00:01<00:00, 159.41it/s]


['name', 'name', 'bear', 'th', 'start', 'tell', 'little', 'bit', 'ancestor', 'come', 'very', 'famous', 'branch', 'jewish', 'religion', 'great', 'grand', 'uncle', 'synagogue', 'name']


In [23]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in tqdm(data_words):
    new = id2word.doc2bow(text)
    corpus.append(new)
print()
print (corpus[10][0:20])

word = id2word[[10][:1][0]]
print (word)

100%|██████████| 252/252 [00:00<00:00, 893.95it/s]


[(1, 4), (10, 1), (12, 1), (20, 5), (21, 1), (24, 7), (25, 2), (26, 1), (33, 2), (38, 2), (39, 2), (54, 2), (56, 5), (60, 1), (61, 1), (65, 3), (66, 18), (67, 2), (78, 6), (79, 2)]
afternoon


In [29]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


## Vizualizing the Data

In [30]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.115828  0.117221       1        1  25.405146
6     -0.038051  0.042145       2        1  20.697256
9      0.030743  0.091561       3        1  18.454695
8     -0.104758  0.066532       4        1  10.105747
2      0.009721  0.154956       5        1   9.516232
5      0.152821  0.094181       6        1   4.566214
1      0.145461 -0.034424       7        1   4.441431
7     -0.122603 -0.045426       8        1   3.851944
3     -0.019033 -0.073474       9        1   2.958364
4      0.061528 -0.413271      10        1   0.002970, topic_info=              Term          Freq         Total Category  logprob  loglift
442           know  20593.000000  20593.000000  Default  30.0000  30.0000
735             so  20490.000000  20490.000000  Default  29.0000  29.0000
131           camp   4375.000000   4375.000000  Default  28.0000  28.0000
2260        ghetto   2553.000000   2553.000000  Default  27.0000  27.0000
801           then   7895.000000   7895.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
442           know      0.001816  20593.700621  Topic10  -9.6337  -5.8197
735             so      0.001813  20490.261232  Topic10  -9.6350  -5.8159
338            get      0.001805  10524.786186  Topic10  -9.6395  -5.1543
11006        armia      0.001804      4.134067  Topic10  -9.6399   2.6876
11023  counterfeit      0.001804      4.134067  Topic10  -9.6399   2.6876

[868 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
5650      1  0.028736  academic
5650      3  0.028736  academic
5650      5  0.086209  academic
5650      6  0.862087  academic
3284      2  0.047324    accent
...     ...       ...       ...
892       8  0.012410      year
892       9  0.009160      year
8659      1  0.126667   yeshiva
8659      4  0.823337   yeshiva
5900      8  0.924972        za

[2395 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 10, 9, 3, 6, 2, 8, 4, 5])